In [ ]:
import os
import pandas as pd

import pandas as pd
import numpy as np
from tqdm import tqdm
import concurrent.futures
import re
from llm_tools.llm_func import do_product_searching

# Loading the file

In [ ]:
# input path
input_path = os.path.join(os.getcwd(), "input/amazon_de_url.xlsx")

file = pd.read_excel(input_path)

# output path 
output_path = 'result.xlsx'
output_path = os.path.join(os.getcwd(), output_path)

# Parameter

In [ ]:
use_match = 0


match_file_path = 'match_file'

if use_match:
    from sentence_transformers import SentenceTransformer
    match_file_path = os.path.join(os.getcwd(), match_file_path)
    match_file = pd.read_excel(match_file_path)
    # make sure the category and web col name unified as well 
    match_file['Category'] = match_file['Category'].apply(lambda x: x.strip())
    # the cate used to do match should be defined as well 

    match_dict = {}
    for i in range(match_file.shape[0]):
        match_dict[match_file.iloc[i, 0]] = list(match_file.iloc[i, 1:])

else:
    # be in the yaml
    web = web
    
    file['web_1'] = web

# matching

In [ ]:
if use_match:
    model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
    def embedding_one(text):
        return model.encode(text)
# the categories associated with files
    file_embedding = {}
    for i in tqdm(set(file['suggest_cate_1']), total=len(set(file['suggest_cate_1']))):
        file_embedding[i] = embedding_one(i)

# the categories associated with the web
    cate_embedding = {}
    for i in tqdm(match_dict, total=len(match_dict)):
        cate_embedding[i] = embedding_one(i)
    cate_list = [i for i in match_dict]
    cate_matrix = np.vstack([cate_embedding[i] for i in cate_list])

    file.reset_index(drop=True, inplace=True)

    cate_col = 'suggest_cate_1'
    n_web = len(match_dict[list(match_dict.keys())[0]])

    results = [[] for i in range(n_web)]
    match_c = []
    sim_v = []
    for i in range(file.shape[0]):
        v = file_embedding[file.loc[i, cate_col]]
    # the similarity
    sim = cate_matrix @ v

    # max similarity
    sim_max_i = np.argmax(sim)

    # matched_cate
    c = cate_list[sim_max_i]

    sim_v.append(sim[sim_max_i])

    match_c.append(c)
    item = match_dict[c]
    for j in range(n_web):
        results[j].append(item[j])

    file['match_c'] = match_c
    file['sim'] = sim_v
    for i in range(n_web):
        file[f"web_{i+1}"] = results[i]   


    ### setting in yaml
    sim_cri = sim_cri
    mask = (file['sim']>sim_cri) 
    # done
    file = file.loc[mask]
    file.reset_index(drop=True, inplace=True)
    

In [ ]:
##### yaml definition
def find_the_url(df, name_col=name_col, web_col='web_1', country=country, url_col_name=url_col_name):
#def find_the_url(df, name_col='item_sku_name_en', web_col='web_1', country='uk', url_col_name='url_search_1'):
  result = []
  df.reset_index(drop=True, inplace=True)
  for i in range(df.shape[0]):
    product_name = df.loc[i, name_col]
    mkt_plc = df.loc[i, web_col]
    r = do_product_searching(product_name, mkt_plc, country=country)
    result.append(r)
  df[url_col_name] = result
  return df